In [1]:
import os
import re
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import boto3
import pyrootutils

pyrootutils.setup_root(os.curdir, indicator=".project-root", pythonpath=True)
from extras import constants, paths
from aws import rds, s3

## 티셔츠 크롤링

### 썸네일 이미지 & 상품 url

In [2]:
page_url = "https://www.slowand.com/category/%ED%8B%B0%EC%85%94%EC%B8%A0/46/?page=1"

In [3]:
response = requests.get(page_url, headers={"User-Agent": "Mozilla/5.0"})
html = response.text
soup = BeautifulSoup(html, "html.parser")

In [4]:
products_li = soup.select('[id^="anchorBoxId_"]')

In [42]:
def size_check(p):
    try:
        if "SIZE" in p.select_one("u").text:
            return True
        elif "FITTING" in p.select_one("u").text:
            return True
        else:
            return False
    except:
        return False
def size_text(soup):
    p_lst = soup.select("div.cont > p")
    idxs = []
    for i, p in enumerate(p_lst):
        if size_check(p):
            idxs.append(i)
            if len(idxs) == 2:
                break
    print(idxs)
    return list(map(lambda x: x.text.strip(), p_lst[idxs[0]+1:idxs[1]]))

In [33]:
product_url

'https://www.slowand.com//product/무료배송질좋은기본티-slowmade-프리미엄-오가닉코튼-무지티셔츠-11-color/4606/category/46/display/2/'

In [43]:
for product_li in products_li:
    product_url = "https://www.slowand.com/" + product_li.select_one("div.thumbnail > div.prdImg > a")["href"]
    print(product_url)
    thumbnail_img_url = product_li.select_one("div.thumbnail > div.prdImg > a > img")["src"]

    response = requests.get(product_url, headers={"User-Agent": "Mozilla/5.0"})
    html = response.text
    soup = BeautifulSoup(html, "html.parser")

    product_dict = {}
    product_dict['price'] = int(re.search(r'\d+', soup.select_one("strong#span_product_price_text").text.strip().replace(",", "")).group())
    product_dict['name'] = soup.select_one("div.infoArea > h2").text.strip()
    product_dict['gender'] = "F"
    product_dict['sub_category_id'] = 9
    product_dict['category_id'] = constants.SUB2CAT[product_dict['sub_category_id']]
    product_dict['url'] = product_url
    product_dict['mall_id'] = constants.SLOWAND_ID
    text = list(filter(lambda x: x.select_one("strong > span").text == '상품간략설명', product_li.select("div.description > ul.xans-element-.xans-product.xans-product-listitem.spec > li")))[0].select("span")[1].find_all(string=True)
    text = '\n'.join(list(map(lambda x: x.strip(), text)))

    print(size_text(soup))

https://www.slowand.com//product/어텀베스트-5무료배송-slowmade-니트라이크-여리핏-골지티셔츠-5-color-프리오더-510일-소요/7720/category/46/display/2/
[63, 66]
['어깨32/가슴34/암홀18/팔기장58/팔단면8.5/소매단면9/밑단36/총장56', '']
https://www.slowand.com//product/무료배송2만장돌파-slowmade-솔트-클린-반팔티셔츠-스탠다드핏루즈핏-7-color-반팔크롭티여름반팔여름티셔츠/7466/category/46/display/2/
[94, 100]
['스탠다드핏-어깨36.5/가슴46/암홀18.5', '팔기장17.5/소매단면15.5/밑단46.5/총장51', '루즈핏-어깨41/가슴52/암홀22.5', '팔기장23/소매단면18/밑단53/총장61.5', '']
https://www.slowand.com//product/가을베스트무료배송-slowmade-여리핏-소프트-골지티셔츠-7-color-프리오더-510일-소요/6801/category/46/display/2/
[64, 68]
['어깨34/가슴44/암홀19/팔기장57/팔단면12.5', '소매단면9.5/밑단44/총장53', '']
https://www.slowand.com//product/무료배송질좋은기본티-slowmade-프리미엄-오가닉코튼-무지티셔츠-11-color/4606/category/46/display/2/
[91, 93]
['어깨50/가슴52.5/암홀23/팔기장53.5/팔단면17.5/소매단면14/밑단52/총장62']
https://www.slowand.com//product/기획특가-slowmade-심플베이직-슬림무지티셔츠-6-color-프리오더-510일-소요/4467/category/46/display/2/
[94, 97]
['어깨36.5/가슴40/암홀20/팔기장61/팔단면11/밑단42/총장64', '']
https://www.slowand.com//product/무료배송1만장돌파-slowmade

In [5]:
product_li = products_li[3]

In [6]:
product_url = "https://www.slowand.com/" + product_li.select_one("div.thumbnail > div.prdImg > a")["href"]
product_url

'https://www.slowand.com//product/가을베스트무료배송-slowmade-여리핏-소프트-골지티셔츠-7-color-프리오더-510일-소요/6801/category/46/display/2/'

In [7]:
thumbnail_img_url = product_li.select_one("div.thumbnail > div.prdImg > a > img")["src"]
thumbnail_img_url

'https://cafe24img.poxo.com/anne2173/web/product/medium/202308/4339c61148378493434728b75094e715.gif'

In [8]:
s3_access_key = pd.read_csv(paths.S3_ACCESS_KEY_PATH)
s3_obj = boto3.client(
    "s3",
    aws_access_key_id=s3_access_key["Access key ID"].values[0],
    aws_secret_access_key=s3_access_key["Secret access key"].values[0],
    region_name="ap-northeast-2",
)

In [9]:
from PIL import Image
response = requests.get(thumbnail_img_url, headers=constants.HEADER)
gif_image = response.content
s3_obj.put_object(
    Bucket="fittering-ml-test-bucket",
    Key="test.gif",
    Body=gif_image,
    ContentType="image/jpg",
)

{'ResponseMetadata': {'RequestId': 'R8QN0SBMPAXH0G1D',
  'HostId': 'P0YFnvAmqoLqSvDM3umATDtgH8K33qmVAJFCM4fPBICqM7gXKelzpqll5niY1EtfRhIqVVJk8hU=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'P0YFnvAmqoLqSvDM3umATDtgH8K33qmVAJFCM4fPBICqM7gXKelzpqll5niY1EtfRhIqVVJk8hU=',
   'x-amz-request-id': 'R8QN0SBMPAXH0G1D',
   'date': 'Tue, 05 Sep 2023 12:40:59 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c3a93af0e1a1794055d81753a0c0c2bd"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c3a93af0e1a1794055d81753a0c0c2bd"',
 'ServerSideEncryption': 'AES256'}

In [10]:
test_gif = Image.open(
    s3_obj.get_object(Bucket="fittering-ml-test-bucket", Key="test.gif")["Body"]
)

In [11]:
imgs = []
for frame_n in range(test_gif.n_frames):
    test_gif.seek(frame_n)
    imgs += [test_gif.convert("RGB")]

### 상품 상세 정보

In [12]:
response = requests.get(product_url, headers={"User-Agent": "Mozilla/5.0"})
html = response.text
soup = BeautifulSoup(html, "html.parser")
product_dict = {}

#### product price

In [13]:
product_dict['price'] = int(re.search(r'\d+', soup.select_one("strong#span_product_price_text").text.strip().replace(",", "")).group())

#### product name

In [14]:
product_dict['name'] = soup.select_one("div.infoArea > h2").text.strip()

#### product gender

In [15]:
product_dict['gender'] = "F"

#### product subcategory_id

In [16]:
product_dict['sub_category_id'] = 9

#### product category_id

In [17]:
product_dict['category_id'] = constants.SUB2CAT[product_dict['sub_category_id']]

#### product url

In [18]:
product_dict['url'] = product_url

#### product mall_id

In [19]:
product_dict['mall_id'] = constants.SLOWAND_ID

#### product text

In [20]:
products_li[0].select("div.description > ul.xans-element-.xans-product.xans-product-listitem.spec > li")[1].select_one("strong > span").text

'상품간략설명'

In [21]:
text = list(filter(lambda x: x.select_one("strong > span").text == '상품간략설명', product_li.select("div.description > ul.xans-element-.xans-product.xans-product-listitem.spec > li")))[0].select("span")[1].find_all(string=True)
text = '\n'.join(list(map(lambda x: x.strip(), text)))
print(text)

많은 분들의 최애티셔츠♥ 재입고!
어깨를 타고 툭 흐르듯 떨어지는 소재감에
가장 여리여리하게 슬림해 보이는 핏감
소장가치 있는 색감들을 담았어요!


In [22]:
with open("test.txt", "w") as f:
    f.write(soup.prettify())

In [23]:
def size_check(p):
    try:
        if "SIZE" in p.select_one("span > font > strong > u").text:
            return True
        elif "FITTING" in p.select_one("span > font > strong > u").text:
            return True
        else:
            return False
    except:
        return False

In [24]:
def size_text(soup):
    p_lst = soup.select("div.cont > p")
    idxs = []
    for i, p in enumerate(p_lst):
        if size_check(p):
            idxs.append(i)
            if len(idxs) == 2:
                break
    print(idxs)
    return list(map(lambda x: x.select_one("span").text.strip(), p_lst[idxs[0]+1:idxs[1]]))

In [25]:
size_text(soup)

[64, 68]


['어깨34/가슴44/암홀19/팔기장57/팔단면12.5', '소매단면9.5/밑단44/총장53', '']